In [1]:
from datetime import datetime
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
from elasticsearch.helpers import bulk
es = Elasticsearch(['atlas-kibana.mwt2.org:9200'])

In [2]:
ind = "network_weather_2-2016.7.14"

my_query = \
{
  "size": 0,
  "aggs": {      
        "type_counts": {
            "terms": { "field": "_type" },
            "aggs": {      
                "src_counts": {
                     "terms": { "field": "src" , "size": 10000},
                     "aggs": {      
                        "dest_counts": {
                        "terms": { "field": "dest" , "size": 10000}
                        }
                     }
                  }
              }
        }
  }
}

In [3]:
response = es.search(index=ind, body=my_query, request_timeout=600)

In [ ]:
#print(response['aggregations']['type_counts']['buckets'])


In [ ]:

types = response['aggregations']['type_counts']['buckets']

for t in types:
    print(t['key'])
    for s in t['src_counts']['buckets']:
        for d in s['dest_counts']['buckets']:
            src=s['key']
            dest=d['key']
            print ('s: ',src,'d: ', dest, '\t docs:', d['doc_count'])
            myscroll={
            "size":100000,
#             "fields": ["timestamp", "delay_mean","throughput","packet_loss","src","dest"],
            "fields": ["timestamp"],
            "query":{
                "filtered": {
                    "query": {
                         "match_all": {} 
                        },
                        "filter" : { 
                            "bool" : {
                                "must" : [
                            {"term": {"_type": {"value": t['key'] } } },
                            {"term": {"src":   {"value": src } } },
                            {"term": {"dest":  {"value": dest } } }
                                ]
                            }

                        }
                }
            }
            ,"sort": [{ "timestamp": { "order": "asc" }}]
            }
            
            toDelete={}
#             oldres=0
            oldtime=""
            counter=0
            scroll = scan(es, query=myscroll, index=ind, scroll='120s')
            for res in scroll:
#                 if counter==0: print(res)
                if res['fields']['timestamp'][0]==oldtime:# and res['fields']['src'][0]==src and res['fields']['dest'][0]==dest and res['fields']['packet_loss'][0]==oldres:
                    toDelete[res['_id']]=res['_index']
#                     print(res['fields'])
#                 oldres=res['fields']['packet_loss'][0]
                oldtime=res['fields']['timestamp'][0]
                counter+=1
#                if counter>200:
#                    break
#                 if counter%100==0: 
                
            print ("records:", counter, '\tto delete:',len(toDelete))
            bulk_body = []
            for x in toDelete:
                bulk_body.append('{"delete": {"_id": "'+x+'", "_index":"'+toDelete[x]+'", "_type":"'+t['key']+'"}}')
#             print (bulk_body)
            if len(toDelete):
                es.bulk('\n'.join(bulk_body))

latency
s:  134.158.123.183 d:  85.122.31.74 	 docs: 3352
records: 3352 	to delete: 1
s:  134.158.123.183 d:  134.158.20.192 	 docs: 3328
records: 3328 	to delete: 0
s:  134.158.123.183 d:  192.12.15.26 	 docs: 3328
records: 3328 	to delete: 0
s:  134.158.123.183 d:  141.108.35.18 	 docs: 3315
records: 3315 	to delete: 0
s:  134.158.123.183 d:  192.108.47.12 	 docs: 3315
records: 3315 	to delete: 0
s:  134.158.123.183 d:  130.246.47.129 	 docs: 3302
records: 3302 	to delete: 2
s:  134.158.123.183 d:  81.180.86.64 	 docs: 2883
records: 2883 	to delete: 0
s:  134.158.123.183 d:  194.80.35.169 	 docs: 2872
records: 2872 	to delete: 0
s:  134.158.123.183 d:  144.206.236.189 	 docs: 2708
records: 2708 	to delete: 0
s:  134.158.123.183 d:  145.100.17.8 	 docs: 2703
records: 2703 	to delete: 0
s:  134.158.123.183 d:  193.60.193.3 	 docs: 1136
records: 1136 	to delete: 0
s:  134.158.123.183 d:  130.209.239.124 	 docs: 1003
records: 1003 	to delete: 0
s:  134.158.123.183 d:  129.107.255.29 	 do